## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
# model = keras.applications.Xception(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=input_tensor,
#     pooling=None,
#     classes=10
# )

'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()


94773248/94765736 [==============================] - 2s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
__________________________

In [4]:
print('Model深度：', len(model.layers))


Model深度： 175


## 添加層數

In [9]:
x = model.output

'''可以參考Cifar10實作章節,自行填入'''
x = GlobalAveragePooling2D()(x)
x = Dense(activation="relu", units=128)(x)
x = Dropout(rate=0.1)(x)

predictions = Dense(units=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 179


## 鎖定特定幾層不要更新權重

In [10]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)


## Training

In [12]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.7297 - accuracy: 0.3898
Epoch 2/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.4738 - accuracy: 0.4780
Epoch 3/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.3744 - accuracy: 0.5125
Epoch 4/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.3316 - accuracy: 0.5276
Epoch 5/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.2828 - accuracy: 0.5455
Epoch 6/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.2248 - accuracy: 0.5626
Epoch 7/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.1859 - accuracy: 0.5799
Epoch 8/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.1852 - accuracy: 0.5791
Epoch 9/100
1563/1563 [==============================] - 44s 28ms/step - loss: 1.1166 - accuracy: 0.6053
Epoch 10/100
1563/1563 [==============================]